Here we will be implementing a version of CoCa classifier (the current top performer on ImageNet database) by training it on imagenet imstances of our classes and then asking it to classify our 'strange' images.

In [1]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
!pip install -U -q scikeras
!pip install -U -q pillow==8.4.0
#!pip install -U -q tensorflow
##==2.7.0
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
#from google.colab import files

#files.upload(); # upload your kaggle.json file

#import json

#!mkdir /root/.kaggle/
#!mv kaggle.json /root/.kaggle/kaggle.json
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle config set -n path -v{/content}

In [3]:
#be careful, the data is like 160gb +
#!kaggle competitions download -c imagenet-object-localization-challenge

#CATS AND DOGS
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

#HANDS:
downloaded = drive.CreateFile({'id':'1KcMYcNJgtK1zZvfl_9sTqnyBUTri2aP2'}) 
downloaded.GetContentFile('Hands.zip')

--2022-11-22 11:01:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  29.1MB/s    in 2.3s    

2022-11-22 11:01:16 (29.1 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
import io
import os 
import zipfile

cd_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(cd_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

h_zip = 'Hands.zip'
zip_ref = zipfile.ZipFile(h_zip, 'r')
zip_ref.extractall('/tmp/cats_and_dogs_filtered/train')
zip_ref.close()

#!unzip '/tmp/cats_and_dogs_filtered.zip'
#!unzip Hands.zip -d 'cats_and_dogs_filtered/train'

!ls

Hands.zip  sample_data


In [5]:
#setting up correct directories
import numpy as np
#from image import PIL

#FOR THE MOMENT ALL HANDS AARE IN TRAIN. THIS IS FINE WHATEVER
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

train_hands_dir = os.path.join(train_dir, 'Hands')


# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

#NOTE: atm all hands in train dir so no path here

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total train hand images:', len(os.listdir(train_hands_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))


total training cat images: 1000
total training dog images: 1000
total train hand images: 11076
total validation cat images: 500
total validation dog images: 500


In [6]:
#PLOT SOME OF THE IMAGES
#code followed closely from: https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=4PIP1rkmeAYS
#%matplotlib inline

#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#import pathlib

# Parameters for our graph; we'll output images in a 4x4 configuration
#nrows = 4
#ncols = 4

# Index for iterating over images
#pic_index = 0
#bBaby = io.BytesIo()
# Set up matplotlib fig, and size it to fit 4x4 pics
#fig = plt.gcf()
#fig.set_size_inches(ncols * 4, nrows * 4)

#pic_index += 8
#next_cat_pix = [os.path.join(train_cats_dir, fname) 
#                for fname in os.listdir(train_cats_dir)[pic_index-8:pic_index]]
#next_dog_pix = [os.path.join(train_dogs_dir, fname) 
#                for fname in os.listdir(train_dogs_dir)[pic_index-8:pic_index]]

#for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
#  sp = plt.subplot(nrows, ncols, i + 1)
#  sp.axis('Off') # Don't show axes (or gridlines)

#  img = mpimg.imread(img_path) #EVEN HERE IT ERRORS. CANNOT USE PIL IMAGES FORMAT?????
#  plt.imshow(img)

#plt.show()


In [7]:
#TODO: convert all hand and cat/dog images to same size
import tensorflow
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
from PIL import Image


# All images will be rescaled by 1./255 for rgb
cdGen = ImageDataGenerator(rescale=1./255)

#WAY STATED IN TUTORIAL TO GET ALL FILES IN

#batch size=all our images
batch_size = len(train_cats_dir) + len(train_dogs_dir) + len(train_hands_dir)

train_generator = cdGen.flow_from_directory(
    directory=train_dir,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k,v in train_generator.class_indices.items()]

X, y = train_generator.next()
#X = images
#y = classes

Found 13076 images belonging to 3 classes.


In [8]:

#clean_directory(train_cats_dir)
#clean_directory(train_dogs_dir)
#clean_directory(train_hands_dir)

#x = np.array([np.array(Image.open(fname)) for fname in filelist])
#cnames = os.listdir(train_cats_dir)
#dnames = os.listdir(train_dogs_dir)
#hnames = os.listdir(train_hands_dir)

#Doesn't matter what I do. PIL images do not work
#cats_train = [os.path.join(train_cats_dir, f) for f in cnames]
#hands_train = [os.path.join(train_hands_dir, f) for f in hnames]
#dogs_train = [os.path.join(train_dogs_dir, f) for f in dnames]

#This crashed runtime lol but works in theory
#hands = np.array([np.array(Image.open(fname)) for fname in hands_train])
#cats = np.array([np.array(Image.open(fname)) for fname in cats_train])
#dogs = np.array([np.array(Image.open(fname)) for fname in dogs_train])


In [9]:
!pip install coca-pytorch
!pip install vit-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 454 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 2.1 MB/s 


In [23]:
import torch

# import vision transformer

from vit_pytorch import ViT
from vit_pytorch.extractor import Extractor

vit = ViT(
    image_size = 256,
    patch_size = 1,
    num_classes = 3,
    dim = 512,
    depth = 6,
    heads = 16,
    mlp_dim = 512
)

vit = Extractor(vit, return_embeddings_only = True, detach = False)

# extractor will enable it so the vision transformer returns its embeddings

# import CoCa and instantiate it

from coca_pytorch.coca_pytorch import CoCa

#FIXED: must enable GPU hardware acceleration in runtime settings or this will not work
coca = CoCa(
    dim = 256,                     # model dimension
    img_encoder = vit,             # vision transformer - image encoder, returning image embeddings as (batch, seq, dim)
    image_dim = 512,               # image embedding dimension, if not the same as model dimensions
    num_tokens = 3,                # number of text tokens
    unimodal_depth = 6,            # depth of the unimodal transformer
    multimodal_depth = 6,          # depth of the multimodal transformer
    dim_head = 64,                 # dimension per attention head
    heads = 8,                     # number of attention heads
    caption_loss_weight = 1.,      # weight on the autoregressive caption loss
    contrastive_loss_weight = 1.,  # weight on the contrastive loss between image and text CLS embeddings
).cuda()

#TODO: need to rescale these basically then we're good
text = torch.from_numpy(y).cuda().long()#text = torch.randint(0, 20000, (4, 512)).cuda() .to(device).long()
images = torch.from_numpy(X).cuda().long() #images = torch.randn(4, 3, 256, 256).cuda() torch.from_numpy(X)

#TODO: we fixed cuda problem, now just need to get input shapes right

# train by giving CoCa your text and images with `return_loss = True`

loss = coca(
    text = text,
    images = images,
    return_loss = True  # set this to True to get the full caption + contrastive loss
)

loss.backward()

# TODO ^ train above on imagenet data

logits = coca(
    text = text,
    images = images
) # (4, 512, 20000)

# and the CLIP-like text and image embeddings as

text_embeds, image_embeds = coca(
    text = text,
    images = images,
    return_embeddings = True
) # (4, 512), (4, 512)

RuntimeError: ignored

In [26]:
import torch

# import vision transformer

from vit_pytorch import ViT
from vit_pytorch.extractor import Extractor

vit = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)

vit = Extractor(vit, return_embeddings_only = True, detach = False)

# extractor will enable it so the vision transformer returns its embeddings

# import CoCa and instantiate it

from coca_pytorch.coca_pytorch import CoCa

coca = CoCa(
    dim = 512,                     # model dimension
    img_encoder = vit,             # vision transformer - image encoder, returning image embeddings as (batch, seq, dim)
    image_dim = 1024,              # image embedding dimension, if not the same as model dimensions
    num_tokens = 20000,            # number of text tokens
    unimodal_depth = 6,            # depth of the unimodal transformer
    multimodal_depth = 6,          # depth of the multimodal transformer
    dim_head = 64,                 # dimension per attention head
    heads = 8,                     # number of attention heads
    caption_loss_weight = 1.,      # weight on the autoregressive caption loss
    contrastive_loss_weight = 1.,  # weight on the contrastive loss between image and text CLS embeddings
).cuda()

# mock text and images

text = torch.randint(0, 20000, (4, 512)).cuda()
#text is 512 long by 4 rows
images = torch.randn(4, 3, 256, 256).cuda()
#images is 4 -> long by 3 rows, each of 256x256 dimensions

#think to matrix multiplication: 3x4 X 4x512 = 3x512
print(images)

# train by giving CoCa your text and images with `return_loss = True`

loss = coca(
    text = text,
    images = images,
    return_loss = True  # set this to True to get the full caption + contrastive loss
)

loss.backward()

# do the above for as much text and images...
# then you can get the caption logits as so

logits = coca(
    text = text,
    images = images
) # (4, 512, 20000)

# and the CLIP-like text and image embeddings as

text_embeds, image_embeds = coca(
    text = text,
    images = images,
    return_embeddings = True
) # (4, 512), (4, 512)


tensor([[[[-3.3463e-01, -1.3828e+00,  1.6301e-01,  ..., -6.8082e-01,
            1.0328e+00, -1.7068e+00],
          [ 2.6002e-01, -3.0462e-01, -1.8600e+00,  ..., -4.6944e-01,
           -4.0507e-01,  3.7863e-01],
          [ 6.8310e-02,  2.2696e-01,  5.0837e-01,  ...,  1.6101e+00,
            5.1867e-02, -5.9583e-01],
          ...,
          [ 1.7805e-01,  3.9538e-01,  1.0447e+00,  ...,  1.7420e+00,
           -9.1518e-01, -4.8141e-01],
          [ 7.5213e-01,  5.4313e-03,  8.2938e-01,  ..., -1.1825e+00,
           -4.4107e-01,  4.4723e-02],
          [-5.2621e-01, -1.9041e-01,  5.4493e-01,  ..., -5.1355e-01,
           -3.1809e-01,  8.6003e-01]],

         [[ 1.1269e+00, -1.1397e+00, -6.5762e-01,  ...,  1.4160e+00,
            1.9983e+00, -1.1635e+00],
          [-1.1897e-01, -1.4436e+00,  1.2104e+00,  ..., -1.5964e+00,
            4.2903e-01, -1.1325e+00],
          [-3.2489e-01,  1.3638e-01,  2.0786e-01,  ...,  7.4081e-01,
           -1.0589e+00,  1.7272e-01],
          ...,
     

CITATIONS/REFERENCES:

@article{afifi201911kHands, 
title = {11K Hands: gender recognition and biometric identification using a large dataset of hand images}, author = {Afifi, Mahmoud}, journal = {Multimedia Tools and Applications}, doi = {10.1007/s11042-019-7424-8}, url = {https://doi.org/10.1007/s11042-019-7424-8}, year={2019} 

@inproceedings{Yu2022CoCaCC,
  title   = {CoCa: Contrastive Captioners are Image-Text Foundation Models},
  author  = {Jiahui Yu and Zirui Wang and Vijay Vasudevan and Legg Yeung and Mojtaba Seyedhosseini and Yonghui Wu},
  year    = {2022}
}}

@Lots of code referenced from:
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=TkNQqjZaB4_u